In [572]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
from tqdm import tqdm
from icecream import ic
from dataclasses import dataclass

logging.basicConfig(level=logging.DEBUG)

In [573]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATIX[c1.Index, c2.Index] = DIST_MATIX[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

NUM_CITIES = len(CITIES)
CITIES.head()


,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


## Functions definition

In [574]:

@dataclass
class Individual:
    path: np.ndarray
    cost: float = None

def tsp_cost(tsp):                       # Professor function (not used in slow algo)
    assert tsp[0] == tsp[-1]           

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATIX[c1, c2]
    return tot_cost

# Sum distances, but does't need the last element of the vector to be equal to the first
def tsp_cost_cutTail(tsp: Individual):
    tot_cost = 0
    for i in range(NUM_CITIES):
        tot_cost += DIST_MATIX[tsp.path[i], tsp.path[(i+1)%NUM_CITIES]]
    return tot_cost

def inversion_mutation(tsp: Individual):
    new_tsp = Individual(tsp.path, tsp.cost)
    i, j = np.random.choice(len(new_tsp.path), 2, replace=False)
    if i > j:
        i, j = j, i
    new_tsp.path[i:j] = new_tsp.path[i:j][::-1]
    new_tsp.cost = tsp_cost_cutTail(new_tsp)
    return new_tsp


def inver_over_crossover(parent1: Individual, parent2: Individual):
    new_path = np.atleast_1d(np.array(parent1.path.copy()))
    parent2_path = np.atleast_1d(np.array(parent2.path))

    length = len(new_path)
    
    i = np.random.randint(0, length)
    first_city = new_path[i]
    
    index_in_parent2 = np.where(parent2_path == first_city)[0][0]
    j = (index_in_parent2 + 1) % length  
    second_city = parent2_path[j]
    
    i = np.where(new_path == first_city)[0][0]
    j = np.where(new_path == second_city)[0][0]
    
    if i > j:
        i, j = j, i
    
    new_path[i:j+1] = new_path[i:j+1][::-1]
    
    return Individual(new_path, tsp_cost_cutTail(Individual(new_path)))


# Perform a local search using inversion mutation
def local_search(individual: Individual, num_iterations: int = 10):

    best_path = individual.path.copy()
    best_cost = individual.cost
    
    for _ in range(num_iterations):
        i, j = np.random.choice(len(best_path), 2, replace=False)
        if i > j:
            i, j = j, i
        new_path = best_path.copy()
        new_path[i:j+1] = new_path[i:j+1][::-1]
        new_cost = tsp_cost_cutTail(Individual(new_path))
        
        if new_cost < best_cost:
            best_path, best_cost = new_path, new_cost

    return Individual(best_path, best_cost)




## Fast algorithm

In [575]:
visited = np.full(len(CITIES), False)
dist = DIST_MATIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])

tot_cost = 0
for c1, c2 in zip(tsp, tsp[1:]):
    tot_cost += DIST_MATIX[c1, c2]
logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

DEBUG:root:step: Acheng -> Harbin (33.60km)
DEBUG:root:step: Harbin -> Shuangcheng (53.02km)
DEBUG:root:step: Shuangcheng -> Yushu (61.85km)
DEBUG:root:step: Yushu -> Wuchang (47.68km)
DEBUG:root:step: Wuchang -> Shulan (59.07km)
DEBUG:root:step: Shulan -> Jishu (17.91km)
DEBUG:root:step: Jishu -> Jilin city (50.81km)
DEBUG:root:step: Jilin city -> Jiutai (65.06km)
DEBUG:root:step: Jiutai -> Dehui (43.68km)
DEBUG:root:step: Dehui -> Changchun (78.49km)
DEBUG:root:step: Changchun -> Gongzhuling (59.12km)
DEBUG:root:step: Gongzhuling -> Siping (54.24km)
DEBUG:root:step: Siping -> Liaoyuan (71.76km)
DEBUG:root:step: Liaoyuan -> Meihekou (60.38km)
DEBUG:root:step: Meihekou -> Panshi (55.16km)
DEBUG:root:step: Panshi -> Huadian (56.40km)
DEBUG:root:step: Huadian -> Jiaohe (96.49km)
DEBUG:root:step: Jiaohe -> Dunhua (82.15km)
DEBUG:root:step: Dunhua -> Helong (110.22km)
DEBUG:root:step: Helong -> Longjing (42.88km)
DEBUG:root:step: Longjing -> Yanji (14.70km)
DEBUG:root:step: Yanji -> Tumen 

## Slow algorithm

In [ ]:

SIZE_POPULATION = 100
NO_PROGRESS_LIMIT = 5000


# Generate random population
population = []
for j in range(SIZE_POPULATION - 1):
    new_path = np.random.choice(range(NUM_CITIES), size=NUM_CITIES, replace=False)
    new_path_cost = tsp_cost_cutTail(Individual(new_path, 0))
    population.append(Individual(new_path, new_path_cost))

# Add best path found with fast algorithm
population.append(Individual(path=tsp[0:NUM_CITIES], cost=tsp_cost_cutTail(Individual(tsp[0:NUM_CITIES]))))

population.sort(key=lambda x: x.cost)

# Initialize best individual
best_individual = population[0]

#ic(population[0])






In [577]:

REPS = 100_000                  # Max num of iterations
prob_inver_over = 1             # Initial value of probability
best_cost_unchanged = 0


for i in tqdm(range(REPS)):

    # Select 2 different parents in the top half of the population
    parent1 = np.random.choice(population[:SIZE_POPULATION // 2])
    parent2 = np.random.choice([ind for ind in population[:SIZE_POPULATION // 2] if not np.array_equal(ind.path, parent1.path)])

    offsprings = []
    
    # Decrease probability of crossover every 10_000 iterations
    if i % (REPS // 10) == 0:
        prob_inver_over = max(0.1, prob_inver_over - 0.1)


    # Inver-over crossover or inversion mutation
    if np.random.rand() < prob_inver_over:
        for k in range(int(SIZE_POPULATION/5)):
            sol = inver_over_crossover(parent1, parent2)
            offsprings.append(sol)
    else:
        for j in range(int(SIZE_POPULATION/5)):
            sol = inversion_mutation(parent1)
            offsprings.append(sol)

    population.extend(offsprings)
    population = sorted(population, key=lambda x: x.cost)[:int(SIZE_POPULATION * 0.3)]


    # Local search on the first 10 individuals
    for idx in range(10):
        improved_individual = local_search(population[idx])
        if improved_individual.cost < population[idx].cost:
            population[idx] = improved_individual

    population.sort(key=lambda x: x.cost)


    # Improvement found
    if best_individual.cost > population[0].cost:
        best_individual = population[0]
        best_cost_unchanged = 0 
    else:
        best_cost_unchanged += 1


    # Reset part of the population if there are no improvements for NO_PROGRESS_LIMIT iterations
    if best_cost_unchanged > NO_PROGRESS_LIMIT:
        for _ in range(int(SIZE_POPULATION * 0.2)):
            new_path = np.random.choice(range(NUM_CITIES), size=NUM_CITIES, replace=False)
            new_path_cost = tsp_cost_cutTail(Individual(new_path, 0))
            population.append(Individual(new_path, new_path_cost))


    # After NO_PROGRESS_LIMIT*5 iterations with no improvements -> break
    if best_cost_unchanged > NO_PROGRESS_LIMIT*5:
        print(f"Algoritmo bloccato dopo {i} iterazioni senza miglioramenti.")
        break

ic(best_individual)



 35%|███▌      | 35371/100000 [18:15<35:38, 30.22it/s]  

Algoritmo bloccato dopo 35373 iterazioni senza miglioramenti.


 35%|███▌      | 35373/100000 [18:15<33:21, 32.30it/s]
ic| best_individual: Individual(path=array([258, 317, 574, 200,  43, 659, 259,  88, 423,  84, 624,  34, 428,
                            433,  16, 167, 151, 374, 598, 175, 551, 606, 371, 342, 141, 616,
                            276, 189, 689, 591, 654, 271, 478, 452, 144, 406, 364, 539, 316,
                            390,  13, 318, 176, 613, 236, 436, 291, 549, 287, 249, 721, 252,
                            119, 614, 395, 190, 262, 117, 673, 231, 633, 295, 109, 524, 357,
                            241,  90, 494, 553, 164, 612, 446, 268, 184, 120, 438, 687, 637,
                            538, 488,  11, 568,  89, 473, 684, 566, 619, 351, 652,  46, 102,
                            225, 698,  64, 345,  61, 497, 467, 430,  27, 193, 111, 523,  26,
                            422, 337, 104, 527, 651, 168, 187, 240, 571, 239, 183,  12, 137,
                             96, 579, 655, 191,   7, 207, 595, 352, 419, 123, 315,  37, 572,

Individual(path=array([258, 317, 574, 200,  43, 659, 259,  88, 423,  84, 624,  34, 428,
       433,  16, 167, 151, 374, 598, 175, 551, 606, 371, 342, 141, 616,
       276, 189, 689, 591, 654, 271, 478, 452, 144, 406, 364, 539, 316,
       390,  13, 318, 176, 613, 236, 436, 291, 549, 287, 249, 721, 252,
       119, 614, 395, 190, 262, 117, 673, 231, 633, 295, 109, 524, 357,
       241,  90, 494, 553, 164, 612, 446, 268, 184, 120, 438, 687, 637,
       538, 488,  11, 568,  89, 473, 684, 566, 619, 351, 652,  46, 102,
       225, 698,  64, 345,  61, 497, 467, 430,  27, 193, 111, 523,  26,
       422, 337, 104, 527, 651, 168, 187, 240, 571, 239, 183,  12, 137,
        96, 579, 655, 191,   7, 207, 595, 352, 419, 123, 315,  37, 572,
       664,  22,  45, 297, 621, 118, 482, 476, 713, 289, 456, 626, 217,
       603, 516, 405, 389, 507, 533, 247,  87, 379, 562, 250, 485, 668,
         8, 593, 210, 602, 366,  98, 244, 693, 460, 365, 246, 491, 286,
       107, 138, 382, 672, 292, 605, 401, 570, 4

In [578]:

for i in range(len(best_individual.path) - 1):
    city = best_individual.path[i]
    closest = best_individual.path[i + 1]
    
    logging.debug(
        f"Step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} "
        f"({DIST_MATIX[city, closest]:.2f} km)"
    )

# add last edge
city = best_individual.path[-1]
closest = best_individual.path[0]
logging.debug(
    f"Step: {CITIES.at[city, 'name']} -> {CITIES.at[closest, 'name']} "
    f"({DIST_MATIX[city, closest]:.2f} km)"
)


DEBUG:root:Step: Jiayuguan -> Leizhou (2377.73 km)
DEBUG:root:Step: Leizhou -> Wufeng (897.84 km)
DEBUG:root:Step: Wufeng -> Hengyang (772.36 km)
DEBUG:root:Step: Hengyang -> Buhe (377.54 km)
DEBUG:root:Step: Buhe -> Yongan (691.99 km)
DEBUG:root:Step: Yongan -> Jiazi (367.80 km)
DEBUG:root:Step: Jiazi -> Datong (1925.06 km)
DEBUG:root:Step: Datong -> Pingdingshan (705.07 km)
DEBUG:root:Step: Pingdingshan -> Danshui (1216.63 km)
DEBUG:root:Step: Danshui -> Xuanwei (1115.24 km)
DEBUG:root:Step: Xuanwei -> Beipiao (2305.36 km)
DEBUG:root:Step: Beipiao -> Pingxiang (1695.17 km)
DEBUG:root:Step: Pingxiang -> Puyang (902.29 km)
DEBUG:root:Step: Puyang -> Baicheng (1284.23 km)
DEBUG:root:Step: Baicheng -> Guiyang (2551.52 km)
DEBUG:root:Step: Guiyang -> Gaoyou (1407.62 km)
DEBUG:root:Step: Gaoyou -> Luqiao (501.69 km)
DEBUG:root:Step: Luqiao -> Xiaolan (1035.81 km)
DEBUG:root:Step: Xiaolan -> Haimen (344.57 km)
DEBUG:root:Step: Haimen -> Tumushuke (3953.89 km)
DEBUG:root:Step: Tumushuke -> X